In [1]:
# import modules

import os
import datetime
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

### 1. Basic Processing

In [2]:
# Useful variables & data loading

raw_data_root = '/home/ubuntu/ecg_mount'
proc_data_root = '/home/ubuntu/ecg_hfpef/data/processed_data'
file_root = './fname_score_pair.csv'
crit_file_root = './fname_crit_pair.csv'
keys = ['als', 'fname', 'score']
crit_keys = ['als', 'fname', 'BMI', 'hypertensive', 'afib', 'pulm hyper', 'age', 'pressure']

score_data = pd.read_csv(f'{raw_data_root}/H2FPEF-echo_.csv', encoding='cp949', low_memory=False)
fname_data = pd.read_csv(f'{raw_data_root}/230111_SevMUSE_EKG_MasterTable.csv', low_memory=False)

num_datas = len(score_data)
batch_size = 20000
num_batches = num_datas//batch_size + 1

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/ecg_mount/H2FPEF-echo_.csv'

In [3]:
# Reset CSV File. 
# Please be careful to run the cell. 

if os.path.isfile(file_root):
    os.remove(file_root)

empty = pd.DataFrame(columns = keys)
empty.to_csv(file_root)

In [4]:
# A function that processes single batch
# From mini_score_df, get fname from fname_df, then append into main_df

def dataBatchProcessing(main_df, mini_score_df, fname_df):
    for i in tqdm(range(len(mini_score_df))):
        als = mini_score_df.iloc[i]['AlsUnitNo']
        ecg_fnames = fname_df.query(f'AlsUnitNo=={als}')
        
        # Skip if there is no ecg data, or als is already in main_df
        if len(ecg_fnames) == 0 or als in main_df['als'].tolist():
            continue
        
        # Find ecg filename that has minimum date diffrence. 
        fname_index = None
        echo_date = datetime.datetime.strptime(mini_score_df.iloc[i]['STUDYDATE'], '%Y-%m-%d')
        min_date_diff = 9999999
        for j in range(len(ecg_fnames)):
            # Check if the waveform has shape (5000, 12)
            if ecg_fnames.iloc[j]['waveform_shape'] != '(5000, 12)':
                continue
            # Minimum data difference
            ecg_date = datetime.datetime.strptime(ecg_fnames.iloc[j]['AcqDate'], '%Y-%m-%d')
            date_diff = abs((echo_date - ecg_date).days)
            if date_diff < min_date_diff:
                min_date_diff = date_diff
                fname_index = j
        # Skip if there is no ecg data that matches the waveform shape (5000, 12)
        if fname_index == None:
            continue
                
        # Save and append to the main stream
        fname = ecg_fnames.iloc[fname_index]['fname']
        score = mini_score_df.iloc[i]['score_H2FPEF']
        temp = pd.DataFrame({'als':als, 'fname':fname, 'score':score}, index=[0])
        main_df = pd.concat([main_df, temp], ignore_index=True)
    
    return main_df

In [5]:
# Main processing function.
# divides data into batches and runs dataBatchProcessing for each batch

def main(score, fname, num_data, bs, end_batch, file_root, start_batch=0):
    for i in range(start_batch, end_batch):
        # Get size of the batch for the minibatch index
        if i == end_batch:
            size = num_data%bs
        else:
            size = bs
        
        # Read mainstream from csv
        data = pd.read_csv(file_root)
        mini_batch = score.iloc[i*bs:i*bs+size]
        
        # Process and save to csv
        print(f'Processing batch {i}/{end_batch}')
        data = dataBatchProcessing(data, mini_batch, fname)
        data.to_csv(file_root)
        print(f'Done batch {i} processing\n')

In [6]:
# Run main

main(score_data, fname_data, num_datas, \
     batch_size, num_batches, file_root, start_batch=0)

Processing batch 0/17


100%|█████████████████████████████████████| 20000/20000 [08:43<00:00, 38.19it/s]


Done batch 0 processing

Processing batch 1/17


100%|█████████████████████████████████████| 20000/20000 [07:12<00:00, 46.25it/s]


Done batch 1 processing

Processing batch 2/17


...

Done batch 14 processing

Processing batch 15/17


100%|█████████████████████████████████████| 20000/20000 [10:54<00:00, 30.54it/s]


Done batch 15 processing

Processing batch 16/17


100%|█████████████████████████████████████████| 670/670 [00:21<00:00, 31.50it/s]


Done batch 16 processing



In [3]:
# Remove Addtional terms from batch calculation

drop_columns = ["Unnamed: 0"] + [f"Unnamed: 0.{i}"for i in range(1, num_batches+1)]
data = pd.read_csv(file_root)
new_data = data.drop(columns=drop_columns)
new_data.to_csv(file_root)

### 2. Additional processing (CRIT)

Reprocessing Data
1. Slice data by batchsize, normalize, and re-save as numpy file
2. Save Hfpef scores with each standards

Heavy:2, Hypertensive:1, Artrial Fibrillation:3, 
Pulmonary Hypertnesion:1, Elder:1, Filling Pressure:1

In [2]:
# Useful Variable 2

raw_data_dir = '/home/ubuntu/ecg_mount/220927_SevMUSE_EKG_waveform'
proc_dir = './processed_data'
file_root = './fname_score_pair.csv'
crit_file_root = './fname_crit_pair.csv'
crit_keys = ['batch_no', 'idx', 'als', 'fname', 'score', \
    'BMI', 'afib', 'PASP', 'age', 'EE']

data_pairs = pd.read_csv(file_root)
batch_size = 2048
num_batches = len(data_pairs)//batch_size + 1
batch_idx = [i*batch_size for i in range(num_batches)] + [len(data_pairs)]
num_threads = 16

score_data = pd.read_csv('/home/ubuntu/ecg_mount/H2FPEF-echo_.csv', encoding='cp949', low_memory=False)

def minmax_scaling(data, new_min=0, new_max=1):
    data_min = np.min(data, axis=1, keepdims=True)
    data_max = np.max(data, axis=1, keepdims=True)
    scaled_data = new_min + (data - data_min) * (new_max - new_min) / (data_max - data_min)
    return scaled_data

In [74]:
# Reset crit_CSV File. 
# Please be careful to run the cell. 

if os.path.isfile(crit_file_root):
    os.remove(crit_file_root)

empty = pd.DataFrame(columns = crit_keys)
empty.to_csv(crit_file_root)

In [75]:
# Reprocessing Data
# 1. Slice data by batchsize, and re-save as numpy file
# 2. Save Hfpef scores with each standards
# Heavy:2, Hypertensive:1, Artrial Fibrillation:3, 
# Pulmonary Hypertnesion:1, Elder:1, Filling Pressure:1

def processData(df_line):
    _, df_line = df_line
    als, fname, score = df_line['als'], df_line['fname'], df_line['score']
    
    new_line = pd.DataFrame(columns = crit_keys)
    score_lines = score_data.query(f"AlsUnitNo=={als}")
    for j in range(len(score_lines)):
        line = score_lines.iloc[j]
        if line['score_H2FPEF'] == df_line['score']:
            new_line = pd.DataFrame({
                # Configuration informations
                'als'  : als, 
                'fname': fname, 
                'score': score,
                
                # Get each criteria
                'BMI'  : line['BMI_30'],
                'afib' : line['afib'],
                'PASP' : (1 if line['PASP_35'] == 1.0 else 0),
                'age'  : line['age_60'],
                'EE'   : line['EE_9']
            }, index=[0])
            break
    
    file_root = f'{raw_data_dir}/{fname}'
    ecg_data = pd.read_csv(file_root).to_numpy()
    return ecg_data, new_line


for i in range(num_batches):
    start_t = time.time()
    
    crit_df = pd.read_csv(crit_file_root, index_col=0)
    
    batch_pairs = data_pairs.iloc[batch_idx[i]:batch_idx[i+1]]
    length = len(batch_pairs)
    batch_pairs = batch_pairs.iterrows()
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch_results = list(tqdm(executor.map(processData, batch_pairs), total=length))
        
    ecg_data = []
    for j, (ecg_datum, new_line) in enumerate(batch_results):
        ecg_data.append(ecg_datum)
        new_line['batch_no'] = i
        new_line['idx'] = j
        crit_df = pd.concat([crit_df, new_line], ignore_index=True)
    
    batch_ecg_data = np.stack(ecg_data)
    batch_ecg_data = minmax_scaling(batch_ecg_data)
    np.save(f'{proc_dir}/batch_{i}', batch_ecg_data)
    crit_df.to_csv(crit_file_root)
    
    print(f"Batch {i+1}/{num_batches+1} saved in {start_t-time.time():.2f} sec!")

100%|█████████████████████████████████████████████████████████████| 2048/2048 [00:15<00:00, 135.26it/s]


Batch 1/89 saved in -23.85 sec!


100%|█████████████████████████████████████████████████████████████| 2048/2048 [00:15<00:00, 134.45it/s]


Batch 2/89 saved in -23.50 sec!


100%|██████████████████████████████████████████████████████████████| 2048/2048 [00:26<00:00, 78.75it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [00:28<00:00, 72.39it/s]


Batch 87/89 saved in -44.06 sec!


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3305/3305 [00:45<00:00, 73.06it/s]


Batch 88/89 saved in -70.78 sec!


IndexError: list index out of range

In [12]:
checkpoint = 87
drop_idx = list(range(batch_idx[checkpoint], batch_idx[num_batches]))
crit_df = pd.read_csv(crit_file_root, index_col=0)
crit_df = crit_df.drop(index=drop_idx)
crit_df.to_csv(crit_file_root)

In [13]:
# Reprocessing Data
# 1. Slice data by batchsize, and re-save as numpy file
# 2. Save Hfpef scores with each standards
# Heavy:2, Hypertensive:1, Artrial Fibrillation:3, 
# Pulmonary Hypertnesion:1, Elder:1, Filling Pressure:1

def processData(df_line):
    _, df_line = df_line
    als, fname, score = df_line['als'], df_line['fname'], df_line['score']
    
    new_line = pd.DataFrame(columns = crit_keys)
    score_lines = score_data.query(f"AlsUnitNo=={als}")
    for j in range(len(score_lines)):
        line = score_lines.iloc[j]
        if line['score_H2FPEF'] == df_line['score']:
            new_line = pd.DataFrame({
                # Configuration informations
                'als'  : als, 
                'fname': fname, 
                'score': score,
                
                # Get each criteria
                'BMI'  : line['BMI_30'],
                'afib' : line['afib'],
                'PASP' : (1 if line['PASP_35'] == 1.0 else 0),
                'age'  : line['age_60'],
                'EE'   : line['EE_9']
            }, index=[0])
            break
    
    file_root = f'{raw_data_dir}/{fname}'
    ecg_data = pd.read_csv(file_root).to_numpy()
    return ecg_data, new_line


for i in range(checkpoint, num_batches):
    start_t = time.time()
    
    crit_df = pd.read_csv(crit_file_root, index_col=0)
    
    batch_pairs = data_pairs.iloc[batch_idx[i]:batch_idx[i+1]]
    length = len(batch_pairs)
    batch_pairs = batch_pairs.iterrows()
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch_results = list(tqdm(executor.map(processData, batch_pairs), total=length))
        
    ecg_data = []
    for j, (ecg_datum, new_line) in enumerate(batch_results):
        ecg_data.append(ecg_datum)
        new_line['batch_no'] = i
        new_line['idx'] = j
        crit_df = pd.concat([crit_df, new_line], ignore_index=True)
    
    batch_ecg_data = np.stack(ecg_data)
    batch_ecg_data = minmax_scaling(batch_ecg_data)
    np.save(f'{proc_dir}/batch_{i}', batch_ecg_data)
    crit_df.to_csv(crit_file_root)
    
    print(f"Batch {i+1}/{num_batches} saved in {start_t-time.time():.2f} sec!")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [00:14<00:00, 137.53it/s]


Batch 88/90 saved in -31.57 sec!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1257/1257 [00:09<00:00, 136.00it/s]


Batch 89/90 saved in -19.37 sec!


In [38]:
for i in tqdm(range(num_batches)):
    batch = np.load(f'{proc_dir}/batch_{i}.npy')
    nan_args = np.argwhere(np.isnan(batch))
    if len(nan_args) == 0:
        continue
    for nan_arg in nan_args:
        idx = tuple(nan_arg)
        batch[idx] = 0
    
    np.save(f'{proc_dir}/batch_{i}', batch)
    print(f"For batch no. {i+1}, fixed {len(nan_args)} data")

 12%|██████████████████▊                                                                                                                                     | 11/89 [00:10<01:22,  1.06s/it]

For batch no. 11, fixed 10000 data


 13%|████████████████████▍                                                                                                                                   | 12/89 [00:11<01:33,  1.22s/it]

For batch no. 12, fixed 5000 data


 18%|███████████████████████████▎                                                                                                                            | 16/89 [00:15<01:24,  1.16s/it]

...

 82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                           | 73/89 [05:34<02:41, 10.12s/it]

For batch no. 73, fixed 15000 data


 85%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                      | 76/89 [05:58<01:52,  8.67s/it]

For batch no. 76, fixed 5000 data


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [07:46<00:00,  5.24s/it]


### 3. Utils

In [4]:
# score min max checking

data = pd.read_csv(file_root)
print(min(data['score']), max(data['score']))

0 7


In [70]:
pd.set_option('display.max_columns', 501)
df = score_data.iloc[0:10]
print(df)

   Unnamed: 0       ID    PATIENTNAME   STUDYDATE  REPORTDATE  \
0           1  8707667         CHO CK  2018-08-20  2018-08-20   
1           2  8705075    KIMJAE SOOK  2018-08-09  2018-08-09   
2           3  8680895            KSJ  2018-07-20  2018-07-20   
3           4  8628848            ALI  2018-04-06  2018-04-06   
4           5  8547639            우원식  2017-11-15  2017-11-15   
5           6  8475444            NaN  2017-08-22  2017-08-22   
6           7  8167661  HWANG UI GOON  2016-02-01  2016-02-01   
7           8  8119372   KIM MYUNG JA  2016-10-25  2016-10-25   
8           9  8079586            염동훈  2015-10-14  2015-10-14   
9          10  8069816        CHOI JS  2015-09-11  2015-09-11   

   VTR.No..Echo.No. VTR.No..Echo.No...1.  Height  Weight   BSA    SBP   DBP  \
0              18.0                  NaN   173.0    59.0  1.68  134.0  74.0   
1              18.0                  NaN   156.0    60.0  1.61  148.0  72.0   
2              18.0                  NaN   165.

In [14]:
crit_file_root = './fname_crit_pair.csv'
df = pd.read_csv(crit_file_root, index_col=0)
df = df.query("batch_no==0")
df = df[['BMI', 'afib']]
print(df)

[[0 0]
 [0 0]
 [0 0]
 ...
 [1 0]
 [0 1]
 [0 0]]
